In [2]:
%reload_ext autoreload
%autoreload 2

In [114]:
import torch
from torch import nn
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from timm.utils.lightning_train import LitTimm # WARNING: This file is COPIED from the one used for actual training
from pathlib import Path

In [16]:
# logdir = Path("/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/")
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT01/tavt_newatt_nobiases_weightsum/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)

In [17]:
model.model.blocks.block.attn.weight_sum

Parameter containing:
tensor([-0.2025, -0.3117, -0.2029, -0.1456, -0.2612, -0.1953, -0.2297, -0.5367,
        -0.1950, -0.2246, -0.6338, -0.2383], requires_grad=True)

Analysis plan

- Analyze the weighted sums trained over the weekend
- Compare with K=V , norm of the value matrix
- Check values of kqt1

I will need 2 that work really well, and 2 that kinda work but are our method

What if one (both) of the energy terms are supposed to be negative?
    - Wouldn't the model just learn the negative values themselves?



In [44]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg4clipinit-True/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.clip(0.0001)

tensor([1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 8.9605e-01,
        1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04],
       grad_fn=<ClampBackward1>)

In [45]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg4clipinit-False/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.clip(0.0001)

tensor([1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04,
        1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04, 1.0000e-04],
       grad_fn=<ClampBackward1>)

In [15]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg3clipinit-False/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.max()

tensor(0.4455, grad_fn=<MaxBackward1>)

In [14]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg3clipinit-True/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.max()

tensor(0.4931, grad_fn=<MaxBackward1>)

In [35]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg4clipinit-False/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.max()

tensor(9.5629e-05, grad_fn=<MaxBackward1>)

In [36]:
model.model.blocks.block.attn.betas

Parameter containing:
tensor([0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250,
        0.1250, 0.1250, 0.1250])

In [42]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg3clipinit-False/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.clip(0.001)

tensor([0.0010, 0.0010, 0.0010, 0.4455, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010,
        0.0010, 0.0010, 0.0010], grad_fn=<ClampBackward1>)

In [48]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg5clipinit-True/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.clip(0.00001)

tensor([1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05,
        1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
       grad_fn=<ClampBackward1>)

In [49]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT01/tavt_newatt_nobiases_hmix/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
# model.model.blocks.block.attn.weight_sum.clip(0.001)

In [53]:
W = model.model.blocks.block.attn.headmixer.weight

In [54]:
A = W @ W.T

In [59]:
evals, evecs = torch.linalg.eigh(A)

In [60]:
evals

tensor([-3.4453e-08, -1.0356e-08, -6.9687e-09, -3.3044e-09, -3.8899e-10,
         1.4360e-09,  3.6544e-09,  1.0649e-08,  1.4137e-08,  1.8588e-08,
         8.0763e-08,  9.2467e-01], grad_fn=<LinalgEighBackward>)

In [64]:
x = torch.randn(12)

In [65]:
A @ x

tensor([-0.0131, -0.0397,  0.0356, -0.0229, -0.0115,  0.0371, -0.0152,  0.0303,
        -0.0192, -0.0403, -0.0186, -0.0517], grad_fn=<MvBackward>)

In [82]:
v = evecs[:,-1]

In [ ]:
evals[-1] * v - (A @ v)

tensor([ 0.0000e+00,  8.9407e-08,  2.9802e-08,  1.4901e-08,  1.4901e-08,
        -2.9802e-08,  2.9802e-08, -2.9802e-08,  0.0000e+00,  2.9802e-08,
         2.9802e-08,  5.9605e-08], grad_fn=<SubBackward0>)

In [ ]:
evals

tensor([-3.4453e-08, -1.0356e-08, -6.9687e-09, -3.3044e-09, -3.8899e-10,
         1.4360e-09,  3.6544e-09,  1.0649e-08,  1.4137e-08,  1.8588e-08,
         8.0763e-08,  9.2467e-01], grad_fn=<LinalgEighBackward>)

## Checking ortho2

In [149]:
# logdir = Path("/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/")
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT03/tavt_newatt_nobiases_hmix_ortho2/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)

# Check headmixing orthogonality
HM = model.model.blocks.block.attn.headmixer
Lold = HM.parametrizations.weight[0].L
HM.parametrizations.weight[0].L = nn.Parameter(torch.ones_like(Lold))
W = HM.weight
res = W @ W.T # Close to identity
print(torch.diag(res))
print(torch.triu(res,1).max())

tensor([0.999999, 1.000000, 0.999999, 1.000000, 1.000000, 1.000000, 1.000000,
        1.000000, 1.000000, 1.000000, 1.000000, 1.000001],
       grad_fn=<DiagBackward>)
tensor(2.033608e-07, grad_fn=<MaxBackward1>)


In [151]:
HM.parametrizations.weight[0].Lclip_max

1.0

In [147]:
torch.set_printoptions(precision=6)

Lold

Parameter containing:
tensor([0.049830, 0.350156, 0.350088, 0.351038, 0.350151, 0.346906, 0.830839,
        0.506304, 0.376687, 0.533895, 0.386158, 0.056187], requires_grad=True)

In [144]:
# Or, we can divide by clipped version of L
HM.parametrizations.weight[0].L = Lold
W = HM.weight
lmin = HM.parametrizations.weight[0].Lclip_min
lmax = HM.parametrizations.weight[0].Lclip_max

# Close to identity
res = torch.einsum("ml,l->ml", W, 1/(Lold.clip(lmin, lmax)))
print(torch.diag(res))
print(torch.triu(res,1).max())

tensor([1.2998, 0.9652, 0.9908, 1.0040, 1.0180, 1.0039, 0.7318, 1.0388, 1.0315,
        1.0280, 1.1064, 1.2178], grad_fn=<DiagBackward>)
tensor(0.2303, grad_fn=<MaxBackward1>)


In [152]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT03/tavt_newatt_1head_nobiases_weightsum/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)


In [158]:
model.model.blocks.block.attn.weight_sum

Parameter containing:
tensor([-0.000361], requires_grad=True)